# Proceso de Limpieza de las dos DB

Primero se va a cargar las librerias que se van a utilizar

In [1]:
import pandas as pd
import datetime as dt

Subir el DataFrame

In [ ]:
df = pd.read_csv(r'C:\Users\AUZ\OneDrive - ULEAM\DATA ANALYTICS\PROJECTS\EDUCACION ECUADOR MATRICULAS\salida matricula.csv', 
                 sep=';')

# Asegurarse que los valores sean numeros enteros en estas columnas
df['Total_Estudiantes']=df['Total_Estudiantes'].astype(int)
df['Promovidos']=df['Promovidos'].astype(int)
df['No promovidos']=df['No promovidos'].astype(int)
df['Abandono']=df['Abandono'].astype(int)

Eliminamos columnas que no nos interesan

In [ ]:
df.drop(columns=['Cod_Provincia','Cod_Canton','Cod_Parroquia','Acceso_Edificio'], inplace=True)

Arreglar la columna de Anio_lectivo

In [ ]:
# Separar columna 'Anio_lectivo' en dos columnas: 'Inicio' 'Fin', manteniendo la columna original
df[['Inicio', 'Fin']] = df['Anio_lectivo'].str.split('-', expand=True)
df['Fin']=df['Fin'].str.slice(start=0, stop=4) # Se eliminan los caracteres que no son parte del año

# Eliminar el 'Fin' de la columna 'Anio_lectivo'
df['Anio_lectivo'] = df['Anio_lectivo'].str.slice(start=0, stop=9) 

# Cambiar el nombre de 'Anio_lectivo' a 'Periodo'
df.rename(columns={'Anio_lectivo':'Periodo'}, inplace=True)
# Cambiar el nombre de 'Inicio' a 'Inicio_Periodo'  
df.rename(columns={'Inicio':'Inicio_Periodo'}, inplace=True)
# Cambiar el nombre de 'Fin' a 'Fin_Periodo'
df.rename(columns={'Fin':'Fin_Periodo'}, inplace=True)

## Estandarizacion de Nombres de Instituciones

Primero se vera cuantas combinaciones hay por Codie AMIE y nombre de Institucion

In [ ]:
df[['AMIE','Nombre_Institucion']].value_counts()

Existen 53 mil combinaciones entre codigo AMIE y Nombre de Institucion, se revisara si cada combinacion es de una escuela diferente, o si es la misma escuela con el codigo AMIE con nombres escritos de diferentes maneras

Por lo que se hara un nuevo df con las combinaciones posibles entre estos dos campos para luego contar cuantas ocurrencias tiene, para luego hacer un LEFT JOIN y asi estandarizar los nombres correctos

In [ ]:
# Se estandarizara los nombres de las Instituciones para los cuales sus nombres son diferentes pero tienen el mismo AMIE
mas_freq=df[['AMIE','Nombre_Institucion']].value_counts().reset_index(drop=False)

# Agrupamos por codigo AMIE y tomamos nomas el valor con mas repeticiones
mas_freq=mas_freq.groupby('AMIE').head(1).reset_index(drop=True)

In [ ]:
# Hacemos el LEFT JOIN para quedarnos con los nombres de las instituciones que tienen el mismo AMIE
df2 = pd.merge(df, mas_freq, left_on='AMIE', right_on='AMIE', how='left')

# Convertimos la columna de Nombre_Institucion a la nueva con los nombres actualizados
df2['Nombre_Institucion_x']=df2['Nombre_Institucion_y']

# Arreglamos la tabla, eliminando las columnas de sobra, y poniendo bien el nombre de Nombre_Institucion
df2.rename(columns={'Nombre_Institucion_x':'Nombre_Institucion'}, inplace=True)
df2.drop(columns=['count'], inplace=True)
df2.drop(columns=['Nombre_Institucion_y'], inplace=True)

Revisamos de nuevo cuantas combinaciones de nombres y Codigos AMIE hay

In [ ]:
df2[['AMIE','Nombre_Institucion']].value_counts()

Ahora existen 29 mil coincidencias, lo que quiere decir que existian aproximadamente 23 mil nombres de Instituciones que estaban escritos de manera diferente, sin embargo eran las mismas Instituciones

# Limpieza de valores Atipicos

Aqui queremos quedarnos unicamente con valores que no sean muy atipicos, como si el promedio de colegios tengo 1 mil estudiantes de los cuales 10 son reprobads=os, si existe un colegio con 10mil estudiantes, de los cuales 100 son reprobados, esto cambiaria nuestras medidas como la media

Para obviar los valores muy atipicos de colegios con una cantidad exagerada, vamos a ver un describe de la tabla para encontrar los valores mximos y pensar si son valores razonables en comparacion con los cuartiles de 75% y decidir si sacarlos de la tabla

In [ ]:
df2.describe()

Para trabajar con estos valores, se trabajara con el 98% de los datos, filtrando el resto, ya que los denominameros atipicos, para lo cual se crea una funcion donde se pase la columna del dataframe y esta calcule el percentil .98 y filtre la tabla para que sean valores menores a estos

In [ ]:
def filtro(columna, dataframe):
    z = dataframe[columna].quantile(0.98)
    dataframe = dataframe[dataframe[columna] < z]
    return dataframe

In [ ]:
df2 = filtro('Total_Estudiantes', df2)
df2 = filtro('Promovidos', df2)
df2 = filtro('No promovidos', df2)
df2 = filtro('Abandono', df2)

Ahora contamos con una base de datos mas limpia de valores atipicos, de nombres mal escritos y de una normalizacion de datos.

Por lo que ya se puede pasar a Power Bi para realizar los graficos y las medidas

In [ ]:
df2.to_csv('./SALIDA.csv', sep=';', index=False, encoding='utf-8-sig')

# Limpiar la otra Base de Datos

In [ ]:
Im=pd.read_csv(r'C:\Users\AUZ\OneDrive - ULEAM\DATA ANALYTICS\PROJECTS\EDUCACION ECUADOR MATRICULAS\inicio matricula.csv',
               sep=';')

Eliminar los valores nulos

In [ ]:
valores_nulos = Im[Im['AMIE'].isnull()].index
Im.drop(index=valores_nulos, inplace=True)

# Quitar las columnas inncesarias

In [ ]:
Im.drop(columns=['Cod_Provincia','Cod_Canton'], inplace=True)

In [ ]:
# Limpiar y convertir las columnas a enteros
columnas = ['Total_Estudiantes', 'Promovidos', 'No promovidos', 'Abandono']
for columna in columnas:
    df[columna] = pd.to_numeric(df[columna], errors='coerce').fillna(0).astype(int)

# Eliminar los valores que no sean numeros enteros

In [ ]:
# Lista de columnas específicas
columnas = ['Ecuatoriana','Colombiana','Venezolana','Peruana','Otros_Paises_de_America','Otros_Continentes' ]

# Convertir las columnas a numéricas y eliminar filas con valores no numéricos
for columna in columnas:
    Im[columna] = pd.to_numeric(Im[columna], errors='coerce')

# Eliminar filas con NaN en las columnas específicas
Im = Im.dropna(subset=columnas)

In [ ]:
Im['Total_Estudiantes']=Im['Total_Estudiantes'].astype(int)
Im['Docentes_Femenino']=Im['Docentes_Femenino'].astype(int)
Im['Docentes_Masculino']=Im['Docentes_Masculino'].astype(int)
Im['Total_Docentes']=Im['Total_Docentes'].astype(int)
Im['Estudiantes_Femenino']=Im['Estudiantes_Femenino'].astype(int)
Im['Estudiantes_Masculino']=Im['Estudiantes_Masculino'].astype(int)
Im['Ecuatoriana']=Im['Ecuatoriana'].astype(int)
Im['Colombiana']=Im['Colombiana'].astype(int)
Im['Venezolana']=Im['Venezolana'].astype(int)
Im['Peruana']=Im['Peruana'].astype(int)
Im['Otros_Paises_de_America']=Im['Otros_Paises_de_America'].astype(int)
Im['Otros_Continentes']=Im['Otros_Continentes'].astype(int)

# Normalizar los nombres de las Instituciones

In [ ]:
mas_freq2=Im[['AMIE','Nombre_Institucion']].value_counts()
mas_freq2=mas_freq2.groupby('AMIE').head(1).reset_index(drop=False)

In [ ]:
Im2=pd.merge(Im, mas_freq2, left_on='AMIE', right_on='AMIE', how='left')
Im2['Nombre_Institucion_x']=Im2['Nombre_Institucion_y']
Im2.rename(columns={'Nombre_Institucion_x':'Nombre_Institucion'}, inplace=True)
Im2.drop(columns=['count'], inplace=True)
Im2.drop(columns=['Nombre_Institucion_y'], inplace=True)


In [ ]:
Im2.Anio_lectivo= Im2.Anio_lectivo.str.slice(start=0, stop=9)

In [ ]:
# Crear columna que funcionara como llave primaria para unirla con la otra tabla
Im2['Indice'] = Im2['AMIE'].astype(str) + ' ' + Im2['Anio_lectivo'].astype(str)
# Crear columna que funcionara como llave primaria para unirla con la otra tabla
df2['Indice'] = df2['AMIE'].astype(str) +' ' + df2['Periodo'].astype(str)

In [ ]:
Im2.to_csv('./INICIO.csv', sep=';', index=False, encoding='utf-8-sig')

In [ ]:
Im2.rename(columns={'Anio_lectivo':'Periodo'}, inplace=True)

In [ ]:
Im2['Zona']=Im2['Zona'].str.slice(start=5, stop=7)

In [ ]:
df2['Zona']=df2['Zona'].str.slice(start=5, stop=7)
df2

In [ ]:
Im2

In [ ]:
# Separar columna 'Periodo' en dos columnas: 'Inicio' 'Fin', manteniendo la columna original
Im2[['Inicio', 'Fin']] = Im2['Periodo'].str.split('-', expand=True)
Im2['Fin']=Im2['Fin'].str.slice(start=0, stop=4) # Se eliminan los caracteres que no son parte del año

# Eliminar el 'Fin' de la columna 'Periodo'
Im2['Periodo'] = Im2['Periodo'].str.slice(start=0, stop=9) 

# Cambiar el nombre de 'Periodo' a 'Periodo'
Im2.rename(columns={'Periodo':'Periodo'}, inplace=True)
# Cambiar el nombre de 'Inicio' a 'Inicio_Periodo'  
Im2.rename(columns={'Inicio':'Inicio_Periodo'}, inplace=True)
# Cambiar el nombre de 'Fin' a 'Fin_Periodo'
Im2.rename(columns={'Fin':'Fin_Periodo'}, inplace=True)

In [ ]:
# Contamos cuántas veces aparece cada combinación AMIE + Nombre
mas_freq = df[['AMIE','Nombre_Institucion']].value_counts().reset_index(drop=False)

# Nos quedamos solo con el nombre más frecuente por cada AMIE
mas_freq = mas_freq.groupby('AMIE').head(1).reset_index(drop=True)

# Hacemos LEFT JOIN para traer el nombre más frecuente según el código
df2 = pd.merge(df, mas_freq, on='AMIE', how='left')

# Reemplazamos el nombre original por el más frecuente
df2['Nombre_Institucion'] = df2['Nombre_Institucion_y']

# Limpiamos columnas sobrantes
df2.drop(columns=['count', 'Nombre_Institucion_y'], inplace=True)

In [249]:
Im2

,Periodo,Zona,Provincia,Canton,Parroquia,Cod_Parroquia,Nombre_Institucion,AMIE,Tipo_Educacion,Sostenimiento,...,Total_Estudiantes,Ecuatoriana,Colombiana,Venezolana,Peruana,Otros_Paises_de_America,Otros_Continentes,Indice,Inicio_Periodo,Fin_Periodo
0,2009-2010,6,AZUAY,CUENCA,EL SAGRARIO,10104.0,UNIDAD EDUCATIVA PARTICULAR ROSA DE JESUS CORDERO,01B00002,Ordinario,Particular,...,1473,1455,0,0,0,18,0,01B00002 2009-2010,2009,2010
1,2009-2010,6,AZUAY,CUENCA,MONAY,10109.0,CEBCI,01B00010,Ordinario,Particular,...,327,311,3,0,0,9,4,01B00010 2009-2010,2009,2010
2,2009-2010,6,AZUAY,CUENCA,BAÑOS,10151.0,CENTRO EDUCATIVO ROUSSEAU,01B00019,Ordinario,Particular,...,40,39,0,0,0,1,0,01B00019 2009-2010,2009,2010
3,2009-2010,6,AZUAY,CUENCA,BAÑOS,10151.0,COLEGIO INTERCULTURAL BILINGUE DE NARANCAY,01B00020,Ordinario,Fiscal,...,264,264,0,0,0,0,0,01B00020 2009-2010,2009,2010
4,2009-2010,6,AZUAY,CUENCA,CHAUCHA,10153.0,ESCUELA DE EDUCACION BASICA SEIS DE JUNIO,01B00021,Ordinario,Fiscal,...,24,24,0,0,0,0,0,01B00021 2009-2010,2009,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306781,2024-2025,5,SANTA ELENA,LA LIBERTAD,LA LIBERTAD,240250.0,ESCUELA DE EDUCACION BASICA JEAN PIAGET,24H00524,Ordinario,Particular,...,191,188,0,3,0,0,0,24H00524 2024-2025,2024,2025
306782,2024-2025,5,SANTA ELENA,SALINAS,JOSE LUIS TAMAYO,240352.0,ESCUELA DE EDUCACION BASICA MONTESSORI,24H00525,Ordinario,Particular,...,256,255,0,0,0,1,0,24H00525 2024-2025,2024,2025
306783,2024-2025,5,SANTA ELENA,LA LIBERTAD,LA LIBERTAD,240250.0,UNIDAD EDUCATIVA PCEI JAMES SMITHSON,24H00526,Popular Permanente,Particular,...,356,348,2,1,0,2,3,24H00526 2024-2025,2024,2025
306784,2024-2025,5,SANTA ELENA,SANTA ELENA,SANTA ELENA,240102.0,COLEGIO DE BACHILLERATO REGAMMI,24H00527,Ordinario,Particular,...,12,12,0,0,0,0,0,24H00527 2024-2025,2024,2025
